In [3]:
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
url_base = 'https://sofifa.com/player/'
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu
import ast
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from bs4 import BeautifulSoup
import pickle
import requests
import re
from bs4 import BeautifulSoup
import requests
import plotly.express as px

2025-02-13 14:29:51.308 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
df = pd.read_csv('/Users/kilian/Documents/GitHub/Projet-3/STREAMLIT/BD/players_3120.csv')

df_final = pd.DataFrame(columns = df.columns, index = range(277644 - df['ID'].min()))
p = df['ID'].min()
liste = []

for m in range(len(df_final)):
    liste.append(p+m)

df_liste = pd.DataFrame({'ID' : liste})
df_final['ID'] = df_liste

df = pd.concat([df, df_final])

df = df.drop_duplicates(subset = 'ID', keep = 'first')

df = df.sort_values(by = 'ID')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kilian/Documents/GitHub/Projet-3/STREAMLIT/BD/players_3120.csv'

In [ ]:
df['name'] = df['name'].apply(lambda x : 'Pas encore fait')

In [ ]:
df_1 = df[:27766]
df_2 = df[27766:55532]
df_3 = df[55532:83298]
df_4 = df[83298:111064]
df_5 = df[111064:138830]
df_6 = df[138830:166596]
df_7 = df[166596:194362]
df_8 = df[194362:222128]
df_9 = df[222128:249894]
df_10 = df[249894:]

In [ ]:
def poids(x):
    texte = x
    chiffre = re.findall("\d+", texte)
    resultat = chiffre[0]
    return resultat

def calcul(value):

    if value == []:
        value = value
    else:
        if type(value) == list:
            value = ''.join(value)
        
        if ' ' in value:
            value = value.replace(' ', '')
            
        if '+' in value:
            a, b = value.split('+')
            return int(a) + int(b)
        elif '-' in value:
            a, b = value.split('-')
            return int(a) - int(b)
        
        value = int(value)
    
    return value

def capacite(id):
    
    navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
    url_base = 'https://sofifa.com/player/'

    id = int(id)
    url_finale_title = f'{url_base}{id}'
    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')
    
    if html_title.status_code == 200:

        dico = {}

        # FIN DE CONTRAT

        for balise_parent in soup_title.find_all('div', class_='grid attribute'):
            if 'Contract valid until' in balise_parent.get_text().strip():
                    end_contract = balise_parent.get_text().strip()

        try:
            end_year = max(re.findall('\d{4,}', end_contract))
            end_year = int(end_year)
        except:
            end_year = 'Unknown'
        
        dico.update({'Team & Contract' : end_year})


        # POSITION

        position = []
        for balise_parent in soup_title.find_all('span', class_='pos'):
            position.append(balise_parent.get_text().strip())

        position = position[0]

        dico.update({'Best position' : position})

        # SALAIRE ET COUT TRANSFERT
        
        for balise_parent2 in soup_title.find_all('div', class_='col'):
            if 'Value' in balise_parent2.get_text().strip():
                valeur = balise_parent2.get_text().strip()
            if 'Wage' in balise_parent2.get_text().strip():
                salary = balise_parent2.get_text().strip()

        try:
            valeur = valeur[:-5]
        except:
            valeur = 0
        try:
            salary = salary[:-4]
        except:
            salary = 0
                    
        chiffres = [valeur, salary]
        new_chiffres = []

        for element in chiffres:
            if element != 0:
                if 'M' not in element and 'K' not in element:
                    new_chiffres.append(int(element.replace('€', '')))
                elif 'K' in element and '.' not in element:
                    new_chiffres.append(int(element.replace('K', '000').replace('€', '')))
                elif 'K' in element and '.'  in element:
                    new_chiffres.append(int(element.replace('K', '00').replace('€', '')))
                elif 'M' in element and '.' not in element:
                    new_chiffres.append(int(element.replace('M', '000000').replace('€', '')))
                elif 'M' in element and '.' in element:
                    new_chiffres.append(int(element.replace('M', '00000').replace('.', '').replace('€', '')))

            else:
                new_chiffres.append(element)

        valeur = int(new_chiffres[0])

        salary = int(new_chiffres[1])

        dico.update({'Value': valeur,
                    'Wage': salary})
        
        # NOM
        
        name = ''

        for balise_parent in soup_title.find_all('div', class_='profile clearfix'):
            for balise_parent2 in soup_title.find_all('h1'):
                name += balise_parent2.get_text().strip() + ', '

        name = name[:-2]

        dico.update({'name' : name})

        # INFOS: AGE, TAILLE, POIDS
        
        for balise_parent in soup_title.find_all('div', class_='profile clearfix'):
            infos = balise_parent.get_text().strip()

        taille = re.findall('(\d{3,})cm', infos)
        poids = re.findall('(\d{2,3})kg', infos)
        age = re.findall('(\d{2,})y.o.', infos)

        dico.update({'Height' : taille[0],
                    'Weight' : poids[0],
                    'Age' : age[0]})

        # FOOT

        texte = ''
        for balise_parent in soup_title.find_all('div', class_='col'):
            texte+= balise_parent.get_text().strip()

        foot = re.findall(r'(?<=Preferred foot )\w+', texte)[0]

        dico.update({'foot' : foot})

        # POTENTIAL


        texte = ""
        for balise_parent2 in soup_title.find_all('div', class_= 'grid'):
            if 'Potential' in balise_parent2.get_text().strip():
                texte += balise_parent2.get_text().strip()

        colonnes = ['Potential']
        for element in colonnes:
            debut = "(\d{2,2} )"
            chiffre = re.findall(f"{debut}{element}", texte)
            chiffre = "".join(chiffre)
            dico.update({element : chiffre})

        for element in colonnes:
            if dico[element] == '':
                debut = r'(\d\d [+-]\d)'
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : calcul(chiffre)})

        for element in colonnes:
            if dico[element] == []:
                debut = "(\d{1,1} )"
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : chiffre})

        for element in colonnes:
            if dico[element] == []:
                dico[element] = 0
            elif type(dico[element]) == list:
                dico[element] = int(''.join(dico[element]))

        # OVERALL


        texte = ""
        for balise_parent2 in soup_title.find_all('div', class_= 'grid'):
            if 'Overall rating' in balise_parent2.get_text().strip():
                texte += balise_parent2.get_text().strip()
        
        colonnes = ['Overall rating']

        for element in colonnes:
            debut = "(\d{2,2} )"
            chiffre = re.findall(f"{debut}{element}", texte)
            chiffre = "".join(chiffre)
            dico.update({element : chiffre})

        for element in colonnes:
            if dico[element] == '':
                debut = r'(\d\d [+-]\d)'
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : calcul(chiffre)})

        for element in colonnes:
            if dico[element] == []:
                debut = "(\d{1,1} )"
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : chiffre})

        for element in colonnes:
            if dico[element] == []:
                dico[element] = 0
            # elif ' ' in dico[element] :
            #     dico[element] = dico[element].replace(' ', '')
            if type(dico[element]) == list:
                dico[element] = int(''.join(dico[element]))


        # CAPACITE
        
        colonnes = list(df.columns)[16:-9]

        colonnes.remove('Total skill')
        colonnes.remove('Total movement')
        colonnes.remove('Total power')
        colonnes.remove('Total mentality')
        colonnes.remove('Total defending')
        colonnes.remove('Total goalkeeping')

        url_finale_title = f'{url_base}{id}'
        html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
        html_title2 = html_title.content
        soup_title = BeautifulSoup(html_title2, 'html.parser')

        texte = ''
        for balise_parent in soup_title.find_all('span'):
            texte += (balise_parent.get_text().strip())
        

        for element in colonnes:
            debut = "(\d{2,2})"
            chiffre = re.findall(f"{debut}{element}", texte)
            dico.update({element : chiffre})
            
            if dico[element] == [] or dico[element] == '':
                debut = r'(\d\d[+-]\d)'
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : calcul(chiffre)})
            
            if dico[element] == [] or dico[element] == '':
                debut = "(\d{1,1})"
                chiffre = re.findall(f"{debut}{element}", texte)
                dico.update({element : chiffre})


        for element in colonnes:
            if type(dico[element]) == list:
                dico[element] = ''.join(dico[element])
            if dico[element] != '' and type(dico[element]) != int and len(dico[element]) <= 3:
                dico[element] = int(dico[element])

        return dico

def remplissage(df):
    for n in range(len(df)):
        id = df['ID'].iloc[n]
        dico = capacite(id)
        if type(dico) == dict:
            for element in df.columns:
                if element in dico.keys():
                    if dico['Team & Contract'] == 'Unknown' or dico['Team & Contract'] > 2022:
                        df[element].iloc[n] = dico[element]


    return df

# def remplissage_final(df):

#     df_def = remplissage(df)
#     df_final = df_def
    
#     df_final = df_final[df_final['name'] != 'Pas encore fait']
#     df_final['Best position'] = df_final['Best position'].apply(lambda x : x.replace('RWB', 'RB').replace('LWB', 'LB').replace('ST', 'CF') if type(x) == str else 'Unknown')
#     df_final['Pied droit'] = df_final['foot'].apply(lambda x : 1 if x == 'Right' else (0 if x == 'Left' else 0.5))
#     df_final = df_final.drop(['foot', 'Unnamed: 64'], axis = 1)

#     return df_final

def remplissage_final(df):

    i = 0

    df_final = pd.DataFrame(columns = df.columns)

    while 'Pas encore fait' in df['name'].to_list():
        if i != 2:
            df_def = remplissage(df)
            if 'Pas encore fait' in df_def['name'].to_list():
                df_rempli = df_def[df_def['name'] != 'Pas encore fait']
                df_vide = df_def[df_def['name'] == 'Pas encore fait']
                df = df_vide
                df_final = pd.concat([df_final, df_rempli])
                
            else:
                df_final = pd.concat([df_final, df_def])
                break
        else:
            break
        i += 1
    
    df_final = df_final[df_final['name'] != 'Pas encore fait']
    df_final['Best position'] = df_final['Best position'].apply(lambda x : x.replace('RWB', 'RB').replace('LWB', 'LB')).replace('ST', 'CF')
    df_final['Pied droit'] = df_final['foot'].apply(lambda x : 1 if x == 'Right' else 0)
    df_final = df_final.drop(['foot', 'Unnamed: 64'], axis = 1)

    return df_final

In [ ]:
df_final_1 = remplissage_final(df_1)
df_final_1.to_csv('dataset_a_jour_1.csv')
df_final_1, df1 = 0, 0

In [ ]:
df_final_2 = remplissage_final(df_2)
df_final_2.to_csv('dataset_a_jour_2.csv')
df_final_2, df1 = 0, 0

In [ ]:
df_final_3 = remplissage_final(df_3)
df_final_3.to_csv('dataset_a_jour_3.csv')
df_final_3, df1 = 0, 0

In [ ]:
df_final_4 = remplissage_final(df_4)
df_final_4.to_csv('dataset_a_jour_4.csv')
df_final_4, df1 = 0, 0

In [ ]:
df_final_5 = remplissage_final(df_5)
df_final_5.to_csv('dataset_a_jour_5.csv')
df_final_5, df1 = 0, 0

In [ ]:
df_final_6 = remplissage_final(df_6)
df_final_6.to_csv('dataset_a_jour_6.csv')
df_final_6, df1 = 0, 0

In [ ]:
df_final_7 = remplissage_final(df_7)
df_final_7.to_csv('dataset_a_jour_7.csv')
df_final_7, df1 = 0, 0

In [ ]:
df_final_8 = remplissage_final(df_8)
df_final_8.to_csv('dataset_a_jour_8.csv')
df_final_8, df1 = 0, 0

In [ ]:
df_final_9 = remplissage_final(df_9)
df_final_9.to_csv('dataset_a_jour_9.csv')
df_final_9, df1 = 0, 0

In [ ]:
df_final_10 = remplissage_final(df_10)
df_final_10.to_csv('dataset_a_jour_10.csv')
df_final_10, df1 = 0, 0